# Python Lab 3: CDMA & Spread Spectrum Simulation

**EE 451: Communications Systems**

**Name:** _________________________ **Date:** _____________

---

## Objectives

1. Generate and analyze Pseudo-Noise (PN) sequences
2. Implement Direct Sequence Spread Spectrum (DSSS)
3. Demonstrate CDMA multi-user communication using Walsh codes
4. Measure processing gain and interference rejection

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import hadamard

plt.rcParams['figure.figsize'] = (12, 5)
print("Setup complete!")

## Part 1: PN Sequence Generation (20 points)

### Task 1.1: Generate a PN Sequence Using LFSR

In [ ]:
def generate_pn_sequence(taps, length, seed=None):
    """Generate PN sequence using Linear Feedback Shift Register.
    
    Args:
        taps: List of tap positions (1-indexed)
        length: Length of sequence to generate
        seed: Initial register state (optional)
    """
    n = max(taps)  # Number of register bits
    if seed is None:
        register = [1] * n  # All ones initial state
    else:
        register = list(seed)
    
    sequence = []
    for _ in range(length):
        # Output is the last bit
        output = register[-1]
        sequence.append(output)
        
        # Feedback is XOR of tapped positions
        feedback = 0
        for tap in taps:
            feedback ^= register[tap - 1]
        
        # Shift register
        register = [feedback] + register[:-1]
    
    return np.array(sequence)

# TODO: Generate a maximal-length PN sequence (n=4, taps=[4,3])
# Maximal length = 2^n - 1 = 15
n = 4
taps = [4, 3]
pn_length = 2**n - 1
pn = generate_pn_sequence(taps, pn_length)

# Convert to bipolar (+1/-1)
pn_bipolar = 2 * pn - 1

print(f"PN sequence (binary): {pn}")
print(f"PN sequence (bipolar): {pn_bipolar}")
print(f"Length: {len(pn)}")

# Plot
plt.figure(figsize=(12, 3))
plt.step(range(len(pn_bipolar)), pn_bipolar, where='mid', linewidth=2)
plt.xlabel('Chip Index')
plt.ylabel('Value')
plt.title(f'PN Sequence (n={n}, length={pn_length})')
plt.ylim(-1.5, 1.5)
plt.grid(True, alpha=0.3)
plt.show()

### Task 1.2: Analyze Autocorrelation Properties

In [ ]:
# TODO: Compute autocorrelation of PN sequence
autocorr = np.correlate(pn_bipolar, pn_bipolar, mode='full')
lags = np.arange(-len(pn_bipolar)+1, len(pn_bipolar))

plt.figure(figsize=(12, 4))
plt.stem(lags, autocorr, linefmt='b-', markerfmt='bo', basefmt='k-')
plt.xlabel('Lag (chips)')
plt.ylabel('Autocorrelation')
plt.title('PN Sequence Autocorrelation')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Peak autocorrelation (lag=0): {autocorr[len(pn_bipolar)-1]}")
print(f"Off-peak autocorrelation: {autocorr[len(pn_bipolar)]}")

### Question 1.1 (10 points)

a) What is the peak autocorrelation value? Why is this value equal to the sequence length?

b) What are the off-peak autocorrelation values? Why is this property important for CDMA?

**Your Answer:**

*[Write your answer here]*

## Part 2: Direct Sequence Spread Spectrum (25 points)

### Task 2.1: Spread and Despread a Signal

In [ ]:
# Data to transmit
data_bits = np.array([1, 0, 1, 1, 0])  # 5 bits
data_bipolar = 2 * data_bits - 1

# Spreading code (use shorter PN for clarity)
spreading_code = np.array([1, 1, -1, 1, -1, -1, 1])  # 7 chips
chips_per_bit = len(spreading_code)

# TODO: Spread the data
# Each data bit is multiplied by the entire spreading code
spread_signal = np.array([])
for bit in data_bipolar:
    spread_signal = np.concatenate([spread_signal, bit * spreading_code])

print(f"Data bits (bipolar): {data_bipolar}")
print(f"Spreading code: {spreading_code}")
print(f"Spread signal length: {len(spread_signal)} chips")
print(f"Processing gain: {chips_per_bit}")

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

# Data bits (repeat each for visualization)
data_expanded = np.repeat(data_bipolar, chips_per_bit)
axes[0].step(range(len(data_expanded)), data_expanded, where='mid', linewidth=2)
axes[0].set_ylabel('Value')
axes[0].set_title('Original Data Bits')
axes[0].set_ylim(-1.5, 1.5)
axes[0].grid(True, alpha=0.3)

axes[1].step(range(len(spread_signal)), spread_signal, where='mid', linewidth=1)
axes[1].set_xlabel('Chip Index')
axes[1].set_ylabel('Value')
axes[1].set_title('Spread Signal')
axes[1].set_ylim(-1.5, 1.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Despread the signal (correlate with spreading code)
despread_bits = []
for i in range(len(data_bits)):
    start = i * chips_per_bit
    end = start + chips_per_bit
    chip_segment = spread_signal[start:end]
    
    # Correlate with spreading code
    correlation = np.sum(chip_segment * spreading_code)
    despread_bits.append(correlation)

despread_bits = np.array(despread_bits)
recovered_bits = (despread_bits > 0).astype(int)

print(f"Correlation values: {despread_bits}")
print(f"Original bits:  {data_bits}")
print(f"Recovered bits: {recovered_bits}")
print(f"Errors: {np.sum(data_bits != recovered_bits)}")

### Question 2.1 (10 points)

a) What is the processing gain in this example? How is it calculated?

b) What are the correlation values for bits 0 and 1? Why are they different signs?

**Your Answer:**

*[Write your answer here]*

## Part 3: CDMA with Walsh Codes (30 points)

### Task 3.1: Generate Walsh Codes

In [ ]:
# Generate Walsh codes using Hadamard matrix
n_users = 4
H = hadamard(n_users)
walsh_codes = H  # Each row is a Walsh code

print("Walsh Codes (Hadamard Matrix):")
print(walsh_codes)

# Verify orthogonality
print("\nOrthogonality check (W * W^T):")
print(walsh_codes @ walsh_codes.T)

### Task 3.2: Simulate Multi-User CDMA

In [ ]:
# Each user transmits one bit
user_data = np.array([1, -1, 1, -1])  # Bipolar data for 4 users

# TODO: Spread each user's data with their Walsh code
spread_signals = []
for i, data in enumerate(user_data):
    spread = data * walsh_codes[i]
    spread_signals.append(spread)
    print(f"User {i+1} (data={data:+d}): {spread}")

# TODO: Sum all signals (simulating shared channel)
composite_signal = np.sum(spread_signals, axis=0)
print(f"\nComposite signal: {composite_signal}")

# Plot
fig, axes = plt.subplots(n_users + 1, 1, figsize=(12, 10))

for i in range(n_users):
    axes[i].step(range(len(walsh_codes[i])), spread_signals[i], where='mid', linewidth=2)
    axes[i].set_ylabel(f'User {i+1}')
    axes[i].set_ylim(-2, 2)
    axes[i].grid(True, alpha=0.3)

axes[n_users].step(range(len(composite_signal)), composite_signal, where='mid', linewidth=2, color='red')
axes[n_users].set_ylabel('Sum')
axes[n_users].set_xlabel('Chip Index')
axes[n_users].set_ylim(-5, 5)
axes[n_users].grid(True, alpha=0.3)

plt.suptitle('CDMA: Individual User Signals and Composite')
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Recover each user's data from composite signal
print("Data Recovery from Composite Signal:")
print("=" * 50)

recovered_data = []
for i in range(n_users):
    # Correlate composite with user's Walsh code
    correlation = np.sum(composite_signal * walsh_codes[i])
    recovered_bit = 1 if correlation > 0 else -1
    recovered_data.append(recovered_bit)
    
    print(f"User {i+1}: correlation={correlation:+3d}, recovered={recovered_bit:+d}, original={user_data[i]:+d}")

print("=" * 50)
errors = np.sum(np.array(recovered_data) != user_data)
print(f"Total errors: {errors}/{n_users}")

### Question 3.1 (15 points)

a) Why can multiple users share the same frequency band simultaneously in CDMA?

b) What property of Walsh codes allows perfect separation of users?

c) What happens if the codes are not perfectly orthogonal (e.g., due to timing errors)?

**Your Answer:**

*[Write your answer here]*

## Part 4: Interference Rejection (25 points)

### Task 4.1: Add Narrowband Interference

In [ ]:
# Simulate DSSS with interference
chips_per_bit = 31  # Processing gain
pn_code = generate_pn_sequence([5, 2], chips_per_bit)
pn_code_bipolar = 2 * pn_code - 1

# Transmit data
data = np.array([1, 0, 1, 0, 1])
data_bipolar = 2 * data - 1

# Spread
spread = np.array([])
for bit in data_bipolar:
    spread = np.concatenate([spread, bit * pn_code_bipolar])

# TODO: Add narrowband interference (jammer)
# Interference is a sinusoid at a single frequency
jammer_amplitude = 3  # Strong jammer
t = np.arange(len(spread))
jammer = jammer_amplitude * np.sin(2 * np.pi * 0.1 * t)  # Narrowband interference

received = spread + jammer

# Despread
despread_clean = []
despread_jammed = []
for i in range(len(data)):
    start = i * chips_per_bit
    end = start + chips_per_bit
    
    # Clean signal
    corr_clean = np.sum(spread[start:end] * pn_code_bipolar)
    despread_clean.append(corr_clean)
    
    # Jammed signal
    corr_jammed = np.sum(received[start:end] * pn_code_bipolar)
    despread_jammed.append(corr_jammed)

print(f"Processing gain: {chips_per_bit}")
print(f"Jammer amplitude: {jammer_amplitude}")
print(f"\nDespread values (no jammer):    {despread_clean}")
print(f"Despread values (with jammer):  {[f'{x:.1f}' for x in despread_jammed]}")
print(f"\nOriginal data:  {data}")
print(f"Recovered data: {(np.array(despread_jammed) > 0).astype(int)}")

In [ ]:
# Plot
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

axes[0].step(range(len(spread)), spread, where='mid', linewidth=1)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Spread Signal (no interference)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(range(len(jammer)), jammer, 'r-', linewidth=1)
axes[1].set_ylabel('Amplitude')
axes[1].set_title(f'Narrowband Jammer (amplitude = {jammer_amplitude})')
axes[1].grid(True, alpha=0.3)

axes[2].step(range(len(received)), received, where='mid', linewidth=1)
axes[2].set_ylabel('Amplitude')
axes[2].set_title('Received Signal (spread + jammer)')
axes[2].grid(True, alpha=0.3)

# Show despread values
bit_centers = np.arange(chips_per_bit//2, len(spread), chips_per_bit)
axes[3].stem(bit_centers[:len(data)], despread_jammed, linefmt='b-', markerfmt='bo', basefmt='k-')
axes[3].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Decision threshold')
axes[3].set_xlabel('Chip Index')
axes[3].set_ylabel('Correlation')
axes[3].set_title('Despread Correlation Values')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Question 4.1 (15 points)

a) Why does spread spectrum reject narrowband interference? What happens to the jammer after despreading?

b) How does processing gain relate to interference rejection capability?

c) Name two real-world applications that use spread spectrum for interference rejection.

**Your Answer:**

*[Write your answer here]*

---

## Lab Summary

### Key Concepts

1. **PN Sequences**: Good autocorrelation properties for synchronization
2. **DSSS**: Spreads signal bandwidth for interference rejection
3. **Processing Gain**: G_p = chips/bit, determines interference rejection
4. **CDMA**: Multiple users share bandwidth using orthogonal codes
5. **Walsh Codes**: Perfectly orthogonal codes for user separation

### Submission Checklist

- [ ] All TODO cells completed
- [ ] All questions answered
- [ ] All cells executed
- [ ] Notebook saved

---

*Submit to Brightspace by due date.*